In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras


import glob as glob

2023-05-17 16:58:35.569100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def filtra(lista):
    m3x3 = []
    mNxN = []
    for var in lista:
        if var[2]==(14751,3,3):
            m3x3.append(var)
        else:
            mNxN.append(var)
    return m3x3,mNxN

def desde3x3(m):
    a = np.load(m)
    a_00 = a[:, 0, 0]
    a_01 = a[:, 0, 1]
    a_02 = a[:, 0, 2]
    a_10 = a[:, 1, 0]
    a_11 = a[:, 1, 1]
    a_12 = a[:, 1, 2]
    a_20 = a[:, 2, 0]
    a_21 = a[:, 2, 1]
    a_22 = a[:, 2, 2]
    resultado = np.column_stack((a_00, a_01, a_02, a_10, a_11, a_12, a_20, a_21, a_22))    
    return resultado


def desde3x3_nombre(m):
    nombre = []
    for i in range(3):
        for j in range(3):
            nombre.append(m[0]+"_"+str(i)+str(j))
    return nombre

In [5]:
variables = glob.glob("../data/kepsilon/kepsilon_PHLL_case_1p0*")


vars_1d  = [(variable[40:-4],variable,np.load(variable).shape)for variable in variables if np.load(variable).shape==(14751,)]
vars_Nd = [(variable[40:-4],variable,np.load(variable).shape)for variable in variables if np.load(variable).shape!=(14751,)]

m3x3,mNxN = filtra(vars_Nd)


In [19]:
len(m3x3.columns)

81

In [20]:
len(vars_1d)

21

In [6]:
nombres= list(chain.from_iterable([desde3x3_nombre(m) for m in m3x3]))
m3x3 = np.column_stack([desde3x3(m[1]) for m in m3x3])
m3x3 = pd.DataFrame(m3x3,columns=nombres)


In [7]:
nombres   = [var[0] for var in vars_1d]
archivos = [var[1] for var in vars_1d]

m1d = pd.DataFrame(np.column_stack([np.load(archivo) for archivo in archivos]),columns=nombres)

In [8]:
data = pd.concat([m1d,m3x3],axis=1)
del data["Cx"]
del data["Cy"]
del data["Cz"]
del data["gradpy"]
del data["gradpx"]
del data["gradpz"]

# data.info()

In [9]:
data.columns

Index(['gradky', 'Ux', 'Uy', 'gradkx', 'gradkz', 'Uz', 'wallDistance', 'T_t',
       'epsilon', 'k', 'p', 'T_k', 'DUDty', 'DUDtx', 'DUDtz', 'Ak_00', 'Ak_01',
       'Ak_02', 'Ak_10', 'Ak_11', 'Ak_12', 'Ak_20', 'Ak_21', 'Ak_22',
       'gradU_00', 'gradU_01', 'gradU_02', 'gradU_10', 'gradU_11', 'gradU_12',
       'gradU_20', 'gradU_21', 'gradU_22', 'Akhat_00', 'Akhat_01', 'Akhat_02',
       'Akhat_10', 'Akhat_11', 'Akhat_12', 'Akhat_20', 'Akhat_21', 'Akhat_22',
       'Shat_00', 'Shat_01', 'Shat_02', 'Shat_10', 'Shat_11', 'Shat_12',
       'Shat_20', 'Shat_21', 'Shat_22', 'S_00', 'S_01', 'S_02', 'S_10', 'S_11',
       'S_12', 'S_20', 'S_21', 'S_22', 'R_00', 'R_01', 'R_02', 'R_10', 'R_11',
       'R_12', 'R_20', 'R_21', 'R_22', 'Aphat_00', 'Aphat_01', 'Aphat_02',
       'Aphat_10', 'Aphat_11', 'Aphat_12', 'Aphat_20', 'Aphat_21', 'Aphat_22',
       'Rhat_00', 'Rhat_01', 'Rhat_02', 'Rhat_10', 'Rhat_11', 'Rhat_12',
       'Rhat_20', 'Rhat_21', 'Rhat_22', 'Ap_00', 'Ap_01', 'Ap_02', 'Ap_10',


In [10]:
# Pearson
matrix = data.corr()

pos = matrix['p'].nlargest(4)[1:].index
neg = matrix['p'].nsmallest(3).index
features_P = list(pos) + list(neg)

features_P

['DUDty', 'Ap_20', 'Akhat_12', 'Ap_02', 'epsilon', 'Ux']

In [11]:
# Pearson 8
matrix = data.corr()

pos = matrix['p'].nlargest(11)[1:].index

neg = matrix['p'].nsmallest(10).index
features_P20 = list(pos) + list(neg)

features_P20,len(features_P20)

(['DUDty',
  'Ap_20',
  'Akhat_12',
  'Shat_01',
  'Shat_10',
  'T_k',
  'Akhat_02',
  'T_t',
  'Shat_11',
  'DUDtx',
  'Ap_02',
  'epsilon',
  'Ux',
  'Akhat_21',
  'k',
  'Akhat_20',
  'Shat_00',
  'R_01',
  'gradU_01',
  'gradky'],
 20)

In [12]:
# Kendall
matrix = data.corr(method="kendall")

pos = matrix['p'].nlargest(4)[1:].index

neg = matrix['p'].nsmallest(3).index
features_K = list(pos) + list(neg)

features_K

['gradU_10', 'DUDty', 'Ap_20', 'Ap_02', 'Aphat_02', 'Ux']

In [13]:
# Spearman
matrix = data.corr(method="spearman")

pos = matrix['p'].nlargest(4)[1:].index

neg = matrix['p'].nsmallest(3).index
features_S = list(pos) + list(neg)

features_S

['gradU_10', 'DUDty', 'Ap_20', 'Ap_02', 'Aphat_02', 'Ux']

In [24]:
matrix.p.nlargest(5)

p           1.000000
gradU_10    0.715101
DUDty       0.708394
Ap_20       0.644773
Aphat_20    0.605612
Name: p, dtype: float64

In [14]:
features_P

['DUDty', 'Ap_20', 'Akhat_12', 'Ap_02', 'epsilon', 'Ux']

In [15]:

df_features = data[features_P]
df_target   = data["p"]

scaler_features = preprocessing.MinMaxScaler()
scaler_target   = preprocessing.MinMaxScaler()

df_features = pd.DataFrame(scaler_features.fit_transform(df_features),
                          columns=features_P)
df_target = pd.DataFrame(scaler_target.fit_transform(df_target.values.reshape(-1,1)),
                          columns=["p"])


# Building my NN
keras.backend.clear_session()

#Ux, Uy
input_layer = keras.layers.Input(shape=(6),name="input_layer")

#Hidden layers
hidden1     = keras.layers.Dense(20,name="Hidden1",kernel_initializer="lecun_normal",
                                activation="selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal", activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal", activation = "selu")(hidden2)
hidden4 = keras.layers.Dense(20,name='Hidden4', kernel_initializer="lecun_normal", activation = "selu")(hidden3)
hidden5 = keras.layers.Dense(20,name='Hidden5', kernel_initializer="lecun_normal", activation = "selu")(hidden4)


output_layer = keras.layers.Dense(1,name="output_layer")(hidden5)

model=keras.Model(inputs=[input_layer], outputs=[output_layer])
# model.summary()
# 
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(optimizer,loss="mse",metrics=["mae","mse"])
# 
history = model.fit([df_features],df_target,
                    epochs=200,
                    verbose= 0,
)


predictions = model.predict(df_features).reshape(len(df_features))
truth = df_target['p']


print(abs(predictions-truth).mean()*100, abs(predictions-truth).max()*100)



KeyboardInterrupt: 

In [ ]:
df_features = data[features_K]
df_target   = data["p"]

scaler_features = preprocessing.MinMaxScaler()
scaler_target   = preprocessing.MinMaxScaler()

df_features = pd.DataFrame(scaler_features.fit_transform(df_features),
                          columns=features_K)
df_target = pd.DataFrame(scaler_target.fit_transform(df_target.values.reshape(-1,1)),
                          columns=["p"])


# Building my NN
keras.backend.clear_session()

#Ux, Uy
input_layer = keras.layers.Input(shape=(6),name="input_layer")

#Hidden layers
hidden1     = keras.layers.Dense(20,name="Hidden1",kernel_initializer="lecun_normal",
                                activation="selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal", activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal", activation = "selu")(hidden2)
hidden4 = keras.layers.Dense(20,name='Hidden4', kernel_initializer="lecun_normal", activation = "selu")(hidden3)
hidden5 = keras.layers.Dense(20,name='Hidden5', kernel_initializer="lecun_normal", activation = "selu")(hidden4)


output_layer = keras.layers.Dense(1,name="output_layer")(hidden5)

model=keras.Model(inputs=[input_layer], outputs=[output_layer])
model.summary()
# 
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(optimizer,loss="mse",metrics=["mae","mse"])
# 
history = model.fit([df_features],df_target,
                    epochs=200,
                    verbose= 0,
)


predictions = model.predict(df_features).reshape(len(df_features))
truth = df_target['p']



print(abs(predictions-truth).mean()*100, abs(predictions-truth).max()*100)



In [ ]:
df_features = data[features_S]
df_target   = data["p"]

scaler_features = preprocessing.MinMaxScaler()
scaler_target   = preprocessing.MinMaxScaler()

df_features = pd.DataFrame(scaler_features.fit_transform(df_features),
                          columns=features_S)
df_target = pd.DataFrame(scaler_target.fit_transform(df_target.values.reshape(-1,1)),
                          columns=["p"])


# Building my NN
keras.backend.clear_session()

#Ux, Uy
input_layer = keras.layers.Input(shape=(6),name="input_layer")

#Hidden layers
hidden1     = keras.layers.Dense(20,name="Hidden1",kernel_initializer="lecun_normal",
                                activation="selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal", activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal", activation = "selu")(hidden2)
hidden4 = keras.layers.Dense(20,name='Hidden4', kernel_initializer="lecun_normal", activation = "selu")(hidden3)
hidden5 = keras.layers.Dense(20,name='Hidden5', kernel_initializer="lecun_normal", activation = "selu")(hidden4)


output_layer = keras.layers.Dense(1,name="output_layer")(hidden5)

model=keras.Model(inputs=[input_layer], outputs=[output_layer])
model.summary()
# 
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(optimizer,loss="mse",metrics=["mae","mse"])
# 
history = model.fit([df_features],df_target,
                    epochs=200,
                    verbose= 0,
)


predictions = model.predict(df_features).reshape(len(df_features))
truth = df_target['p']



print(abs(predictions-truth).mean()*100, abs(predictions-truth).max()*100)



In [ ]:
df_features = data[features_P20]
df_target   = data["p"]

scaler_features = preprocessing.MinMaxScaler()
scaler_target   = preprocessing.MinMaxScaler()

df_features = pd.DataFrame(scaler_features.fit_transform(df_features),
                          columns=features_P20)
df_target = pd.DataFrame(scaler_target.fit_transform(df_target.values.reshape(-1,1)),
                          columns=["p"])


# Building my NN
keras.backend.clear_session()

#Ux, Uy
input_layer = keras.layers.Input(shape=(20),name="input_layer")

#Hidden layers
hidden1     = keras.layers.Dense(20,name="Hidden1",kernel_initializer="lecun_normal",
                                activation="selu")(input_layer)
hidden2 = keras.layers.Dense(20,name='Hidden2', kernel_initializer="lecun_normal", activation = "selu")(hidden1)
hidden3 = keras.layers.Dense(20,name='Hidden3', kernel_initializer="lecun_normal", activation = "selu")(hidden2)
hidden4 = keras.layers.Dense(20,name='Hidden4', kernel_initializer="lecun_normal", activation = "selu")(hidden3)
hidden5 = keras.layers.Dense(20,name='Hidden5', kernel_initializer="lecun_normal", activation = "selu")(hidden4)


output_layer = keras.layers.Dense(1,name="output_layer")(hidden5)

model=keras.Model(inputs=[input_layer], outputs=[output_layer])
model.summary()
# 
optimizer = tf.keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(optimizer,loss="mse",metrics=["mae","mse"])
# 
history = model.fit([df_features],df_target,
                    epochs=200,
                    verbose= 0,
)


predictions = model.predict(df_features).reshape(len(df_features))
truth = df_target['p']



print(abs(predictions-truth).mean()*100, abs(predictions-truth).max()*100)



In [ ]:
df_features.iloc[0]

In [32]:
model.predict([df_features.iloc[0:1]])

1/1 [==============================] - 0s 21ms/step


array([[0.65552163]], dtype=float32)

# Predict pressure using any model trainned by you, using 5 specific cases.
* calculate mean absolute error
* calculate maximum error
* plot error as a funciont of points
* Not allowed to use **PHLL_case**


In [ ]:
vars